# Mandelbrot: numpy vs. numba vectorization
----

## Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numba

**Define the escape time function**

In [ ]:
def escape_time(p, maxtime):
    """Perform the Mantelbrot iteration until it's clear that $p$ diverges 
    or the maximum number of iterations has been reached.
    
    Parameters
    ----------
    p: complex
        point in the complex plane
    maxtime: int
        maximum number of iterations to perform
    """
    z = 0j
    for i in range(maxtime):
        z = z ** 2 + p
        if abs(z) > 2:
            return i
    return maxtime

**Setup**

In [ ]:
# Problem size
n = 2056
print(n)
maxtime = 50

# Domain
x = np.linspace(-2.2, 1.5, n)
y = np.linspace(-1.5j, 1.5j, n)
x, y = np.meshgrid(x, y)

# Define complex plane
p = x + y 

## Numpy vectorize

In [ ]:
escape_time_numpy = np.vectorize(escape_time)

In [ ]:
%%time 
t = escape_time_numpy(p, maxtime)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(t, cmap='viridis')

## Numba vectorize (+ multithreaded)

In [ ]:
escape_time_numba = numba.vectorize(['i4(c16,i4)'], nopython=True, target='parallel')(escape_time)

In [ ]:
%%time
t2 = escape_time_numba(p, maxtime)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(t2)